<a href="https://colab.research.google.com/github/DeepOde/tones2tunes/blob/main/models/Model_BI_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn

In [ ]:
class model1(nn.Module):
  def __init__(self,unique_notes,dropout=0.3,output_emb=100,rnn_unit=128, dense_unit=64):
    super().__init__()
    # this indicates the vocabulary(unique notes in the whole input)
    self.num_embeddings=unique_notes+1
    # this indicates the vector representation size for each note
    self.embedding_dim=output_emb
    # No of hidden layers in GRU
    self.hidden_unit=rnn_unit
    # creating embedding layer
    self.embedding=nn.Embedding(self.num_embeddings, self.embedding_dim)
    # creating bidirectional GRU
    # by default pytorch returns all the hidden state repre's
    # for more read this [@ https://discuss.pytorch.org/t/pytorch-equivalent-to-keras-layers-lstm-return-sequences-false/53976/2]
    self.BiGRU1=nn.GRU(self.embedding_dim,self.hidden_unit,bidirectional=True,batch_first=True)
    # have to implement self attention(Have to check)

    # adding dropout
    self.dropout1 = nn.Dropout(dropout)
    # adding BIGRU
    self.BiGRU2=nn.GRU(self.hidden_unit,self.hidden_unit,bidirectional=True,batch_first=True)
    # adding dropout
    self.dropout2 = nn.Dropout(dropout)
    # adding fully connected layer
    self.fc = nn.Linear(self.hidden_unit, unique_notes+1)
    self.softmax = nn.Softmax(dim=-1)
  def forward(self,x):
    x=self.embedding(x)
    print(x.shape)
    output, ht=self.BiGRU1(x)
    print(output.shape,ht.shape)
    output=output[:,:,:128]
    output=self.dropout1(output)
    output,ht =self.BiGRU2(output)
    print(output.shape,ht.shape)
    output=ht[-1]
    output=self.dropout2(output)
    output=self.fc(output)
    output=self.softmax(output)
    return output   
    

In [ ]:
model=model1(100)

In [ ]:
from torchsummary import summary

In [ ]:
data=torch.randint(10,100,size=(10,1000))


In [ ]:
model.forward(data).shape

torch.Size([10, 1000, 100])
torch.Size([10, 1000, 256]) torch.Size([2, 10, 128])
torch.Size([10, 1000, 256]) torch.Size([2, 10, 128])


tensor([[0.0083, 0.0106, 0.0099,  ..., 0.0122, 0.0112, 0.0085],
        [0.0107, 0.0113, 0.0112,  ..., 0.0124, 0.0117, 0.0090],
        [0.0096, 0.0089, 0.0105,  ..., 0.0100, 0.0109, 0.0089],
        ...,
        [0.0091, 0.0102, 0.0109,  ..., 0.0110, 0.0105, 0.0084],
        [0.0091, 0.0094, 0.0099,  ..., 0.0092, 0.0116, 0.0103],
        [0.0095, 0.0098, 0.0094,  ..., 0.0114, 0.0096, 0.0096]],
       grad_fn=<SoftmaxBackward>)

In [ ]:
o,h=BiGRU1(torch.randn(10,50,100))

In [ ]:
o[:,:,:128].shape

torch.Size([10, 50, 128])

In [ ]:
h.shape

torch.Size([2, 10, 128])